## Continu Inzicht Demo (Create DB tables)
> https://tscontinuinzicht.hkvservices.nl/ci-demo

**Imports**

In [ ]:
# import algemeen
from pathlib import Path

# imports base modules
from toolbox_continu_inzicht import Config, DataAdapter

**Inlezen configuratie**
<details>
<summary>Configuratie bekijken</summary>   

<pre style="font-size: 10pt">
GlobalVariables:
    rootdir: "data"

DataAdapter:
    default_options:
        csv:
            sep: ","

    in_dikesystems_geojson:
        type: shape
        path: "dikesystems.geojson"

    out_dikesystems_table:
        type: ci_postgresql_dikesystem
        database: "continuinzicht"
        schema: "continuinzicht_demo_whatif"

    in_segments_geojson:
        type: shape
        path: "segments.geojson"

    out_segments_table:
        type: ci_postgresql_segment
        database: "continuinzicht"
        schema: "continuinzicht_demo_whatif"

    in_sections_geojson:
        type: shape
        path: "sections.geojson"

    out_sections_table:
        type: ci_postgresql_section
        database: "continuinzicht"
        schema: "continuinzicht_demo_whatif"

    in_measuringstations_csv:
        type: csv
        path: "measuringstations.csv"

    out_measuringstations_table:
        type: ci_postgresql_measuringstation
        database: "continuinzicht"
        schema: "continuinzicht_demo_whatif"

    in_conditions_csv:
        type: csv
        path: "conditions.csv"

    out_conditions_table:
        type: ci_postgresql_conditions
        database: "continuinzicht"
        schema: "continuinzicht_demo_whatif"

    in_sectionfractions_csv:
        type: csv
        path: "sectionfractions.csv"

    out_sectionfractions_table:
        type: ci_postgresql_sectionfractions
        database: "continuinzicht"
        schema: "continuinzicht_demo_whatif"

    in_scenarios_csv:
        type: csv
        path: "scenarios.csv"

    out_scenarios_table:
        type: ci_postgresql_to_scenarios
        database: "continuinzicht"
        schema: "continuinzicht_demo_whatif"

    in_loads_csv:
        type: csv
        path: "loads.csv"

    out_loads_table:
        type: ci_postgresql_to_load
        database: "continuinzicht"
        schema: "continuinzicht_demo_whatif"

    in_fragilitycurves_csv:
        type: csv
        path: "fragilitycurves.csv"

    out_fragilitycurves_table:
        type: ci_postgresql_to_fragilitycurves
        database: "continuinzicht"
        schema: "continuinzicht_demo_whatif"

</pre>
</details>

In [ ]:
# lees configuratiebestand
yaml_config_file = "continu-inzicht-demo-create-db.yaml"
data_path = Path.joinpath(Path.cwd(), "data", yaml_config_file)

config = Config(config_path=data_path)
config.lees_config()

**Aanmaken adapter**

In [ ]:
# data adapter aanmaken
data_adapter = DataAdapter(config=config)

**Waterkeringsystemen inlezen uit geosjon-bestand en wegschrijven naar de database**

In [ ]:
# lees de waterkeringsystemen uit een geosjon-bestand via de data_adapter 'in_dikesystems_geojson'
df_waterkeringsystemen = data_adapter.input(input="in_dikesystems_geojson")

# sla de waterkeringsystemen op in de CI database (tabel 'dikesystems') via de data_adapter 'out_dikesystems_table'
data_adapter.output(output="out_dikesystems_table", df=df_waterkeringsystemen)

**Segmenten inlezen uit geosjon-bestand en wegschrijven naar de database**

In [ ]:
# lees de segmenten uit een geosjon-bestand via de data_adapter 'in_segments_geojson'
df_segmenten = data_adapter.input(input="in_segments_geojson")

# sla de segmenten op in de CI database (tabel 'segments') via de data_adapter 'out_segments_table'
data_adapter.output(output="out_segments_table", df=df_segmenten)

**Secties inlezen uit geosjon-bestand en wegschrijven naar de database**

In [ ]:
# lees de secties uit een geosjon-bestand via de data_adapter 'in_sections_geojson'
df_secties = data_adapter.input(input="in_sections_geojson")

# sla de secties op in de CI database (tabel 'sections') via de data_adapter 'out_sections_table'
data_adapter.output(output="out_sections_table", df=df_secties)

**Meetlocaties inlezen uit CSV en wegschrijven naar de database**

In [ ]:
# lees meetlocaties uit een CSV bestand via de data_adapter 'in_measuringstations_csv'
df_measuringstations = data_adapter.input(input="in_measuringstations_csv")

# sla de meetlocaties op in de CI database (tabel 'measuringstations') via de data_adapter 'out_measuringstations_table'
data_adapter.output(output="out_measuringstations_table", df=df_measuringstations)

**Condities van meetlocaties inlezen uit CSV en wegschrijven naar de database**

In [ ]:
# lees condities van meetlocaties uit een CSV bestand via de data_adapter 'in_conditions_csv'
df_conditions = data_adapter.input(input="in_conditions_csv")

# sla de condities van meetlocaties op in de CI database (tabel 'conditions') via de data_adapter 'out_conditions_table'
data_adapter.output(output="out_conditions_table", df=df_conditions)

**(Interpolatie)fracties voor belastingen per vak inlezen uit CSV en wegschrijven naar de database**

In [ ]:
# lees (interpolatie)fracties meetlocaties per sectie uit een CSV bestand via de data_adapter 'in_sectionfractions_csv'
df_sectionfractions = data_adapter.input(input="in_sectionfractions_csv")

# sla de (interpolatie)fracties meetlocaties per sectie op in de CI database (tabel 'sectionfractions') via de data_adapter 'out_sectionfractions_table'
data_adapter.output(output="out_sectionfractions_table", df=df_sectionfractions)

**What-if scenario's inlezen uit CSV en wegschrijven naar de database**

In [ ]:
# lees de what-if scenario's uit een CSV bestand via de data_adapter 'in_scenarios_csv'
df_what_if_scenarios = data_adapter.input(input="in_scenarios_csv")

# sla de what-if scenarios op in de CI database (tabel 'scenarios') via de data_adapter 'out_scenarios_table'
data_adapter.output(output="out_scenarios_table", df=df_what_if_scenarios)

**What-if belastingen inlezen uit CSV en wegschrijven naar de database**

In [ ]:
# lees de what-if belastingen uit een CSV bestand via de data_adapter 'in_loads_csv'
df_what_if_loads = data_adapter.input(input="in_loads_csv")

# sla de what-if waterstanden op in de CI database (tabel 'waterlevels') via de data_adapter 'out_loads_table'
data_adapter.output(output="out_loads_table", df=df_what_if_loads)

**Fragility curves inlezen uit CSV en wegschrijven naar database**

In [ ]:
# lees de fragility curves uit een CSV bestand via de data_adapter 'in_fragilitycurves_csv'
df_what_if_fragilitycurves = data_adapter.input(input="in_fragilitycurves_csv")

# sla de fragility curves op in de CI database (tabel 'fragilitycurves') via de data_adapter 'out_fragilitycurves_table'
data_adapter.output(output="out_fragilitycurves_table", df=df_what_if_fragilitycurves)